# A FEM implementation of "On a Wildland Fire Model with Radiation" using Firedrake

Asensio, M.I. and Ferragut, L. (2002), On a wildland fire model with radiation. Int. J. Numer. Meth. Engng., 54: 137-157. https://doi.org/10.1002/nme.420

In [32]:
from firedrake import *
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy import constants

In [33]:
# Mesh parameters
nx = 500
ny = 500
Lx = 100
Ly = 100
mesh = RectangleMesh(nx, ny, Lx, Ly)
x, y = SpatialCoordinate(mesh)

In [34]:
# Simulation time parameters
time = 5
num_steps = 100
dt_step = time / num_steps
dt = Constant(dt_step)

In [35]:
# Define the wind field
w_uv = as_vector([100, 0])
W = VectorFunctionSpace(mesh, family='CG', degree=2, dim=2)
w = interpolate(w_uv, W)

In [36]:
# Model Constants
T_pc = Constant(550)  # Remperature Phase change, K
T_inf = Constant(295.372)  # Ambient Temperature, K
rho = Constant(1.1774)  # Atmospheric Density, kg/m^3
C = Constant(1.0057)  # Atmospheric Specific Heat (kJ/kG)K
sigma = Constant(constants.sigma)  # 
delta = Constant(Lx / nx)
H = Constant(15900) # Heat of Combustion of Cellulose, kcal/kg
A = Constant(10 ** 9) # pre-exponential factor, 1/s 
E_A = Constant(83.68) # Activation Energy, kJ/mol
R = Constant(constants.R)
h = Constant(0.1) # Who knows?

In [37]:
# Model Functions
def s(T):
    return conditional(ge(T, T_pc), 1, 0)

In [38]:
# Define the mixed elements
P1 = FiniteElement('P', triangle, 1)
element = MixedElement([P1, P1])

# Define the test functions
V = FunctionSpace(mesh, element)
v_1, v_2 = TestFunctions(V)

# No trial functions since the problem is nonlinear
u = Function(V)
u_n = Function(V)
T, Y = split(u)
T_n, Y_n = split(u_n)

In [39]:
# Initial Conditions
r_center = sqrt((x - Lx / 2) ** 4 + (y - Ly / 2) ** 4)  # radial distance from center of domain
u_max = 10  # Stability is strongly related to this value, make dt smaller if this higher
radius = 4.0
smooth = 1.5
uic = u_max * (1 - 1 / (1 + exp(-smooth * (r_center - radius))))
u.sub(0).assign(project(uic, V[0]))
u.sub(1).assign(project(Constant(1), V[1]))

Coefficient(WithGeometry(IndexedProxyFunctionSpace(<firedrake.mesh.MeshTopology object at 0x15a006820>, FiniteElement('Lagrange', triangle, 1), name=None, index=1, component=None), Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 113)), 282)

In [40]:
# Variational Form
convection = ((rho * C * (((T - T_n)/dt) + inner(w, grad(T))))* v_1) * dx
radiation = (inner((Constant(4) * sigma * delta * pow(T, 3)) * grad(T), grad(v_1))) * dx
reaction = (s(T) * H * A * exp(-E_A/(R * T)) * rho * Y * v_1) * dx
natural_convection = (h * (T - T_inf) * v_1) * dx

mass_frac = (((Y - Y_n)/dt) * v_2) * dx
second_reaction = (s(T) * Y * A * exp(-E_A/(R * T)) * v_2) * dx

F = convection - radiation - reaction + natural_convection + mass_frac + second_reaction 

In [41]:
solver_parameters = {
    "mat_type": "aij",
    "ksp_type": "preonly",
    # Use MUMPS since it handles the null space
    "pc_type": "lu",
    "pc_factor_mat_solver_type": "mumps"
}

problem = NonlinearVariationalProblem(F, u_n)
solver = NonlinearVariationalSolver(problem, solver_parameters=solver_parameters)


# Initialize first time step
t = 0

# Initialize output files
vtkfile_T = File('feragut_output/T.pvd')
vtkfile_Y = File('feragut_output/Y.pvd')
_T, _Y = u.split()
vtkfile_T.write(_T, time=t)
vtkfile_Y.write(_Y, time=t)

# Loop over all time steps
for n in tqdm(range(num_steps)):
    t += dt_step

    # Solve the PDE
    # solve(F == 0, u)
    solver.solve()

    # Assign the variables to the next time steps
    u.assign(u_n)

    # Save solution to file (VTK)
    _T, _Y = u.split()
    vtkfile_T.write(_T, time=t)
    vtkfile_Y.write(_Y, time=t)
        


  4%|▍         | 4/100 [00:28<11:29,  7.18s/it]


ConvergenceError: Nonlinear solve failed to converge after 0 nonlinear iterations.
Reason:
   DIVERGED_FNORM_NAN